In [1]:
import os
import gsw
import datetime
import numpy as np
import numpy as np
import pandas as pd
import seawater as sw
#import cmocean.cm as cmo
import matplotlib.pyplot as plt

/var/folders/n4/nk6k_c65111dd1zq088dqpb40000gn/T/ipykernel_41385/3480337663.py:7: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw


In [2]:
print(os.getcwd())
new_folder = "/Users/sydneyperkins/miniconda3/envs/heatfluxcode/code_files"
os.chdir(new_folder)
print("Current directory set to:", os.getcwd())

/Users/sydneyperkins/miniconda3/envs/heatfluxcode/code_files
Current directory set to: /Users/sydneyperkins/miniconda3/envs/heatfluxcode/code_files


In [3]:
from glob import glob
from scipy.io import loadmat
#from pymatreader import read_mat
from mpl_toolkits import mplot3d
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from scipy.ndimage import gaussian_filter1d as gfilter
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
mat_data = loadmat('/Users/sydneyperkins/Downloads/ASTRAL_DBASIS_upper_CTDgrid.mat')
key = list(mat_data.keys())[-1]
data = mat_data[key][0]
time = data['std_profiles'][0]['time'][0][0].squeeze()
time = time[2:] # Removing areas with no data in the vector\n",
time = pd.to_datetime(time-719529, unit='D') # transforming the matlab time format (datenum) to the python time format (datetime)
time = np.delete(time,[927, 3780, 3781], 0) #deleting bad profiles
depth = data['std_profiles'][0]['P'][0][0];
depth = depth[20:381,2:] # Removing areas with no data
depth = np.delete(depth,[927, 3780, 3781], 1) #deleting bad profiles
vmin, vmax = np.round(np.nanmin(depth),2), np.round(np.nanmax(depth),2)
delta = .25
depth = np.arange(vmin, vmax + delta, delta) #recreating a depth vector that makes sense
time, depth = np.meshgrid(time, depth)
rho = data['std_profiles'][0]['rho'][0][0]; rho = rho[20:381,2:] #potential density
rho = np.delete(rho,[927, 3780, 3781], 1) #deleting bad profiles
sigma = rho-1e3
insitutemp = data['std_profiles'][0]['T'][0][0]; insitutemp = insitutemp[20:381,2:] # in-situ temperature
insitutemp = np.delete(insitutemp,[927, 3780, 3781], 1) #deleting bad profiles
SP = data['std_profiles'][0]['S'][0][0]; SP = SP[20:381,2:] #practical salinity
SP = np.delete(SP,[927, 3780, 3781], 1) #deleting bad profiles
P = data['std_profiles'][0]['P'][0][0]; P = P[20:381,2:] #Pressure
P = np.delete(P,[927, 3780, 3781], 1) #deleting bad profiles
SA = gsw.SA_from_SP(SP, depth, 85, 12) # computing absolutesalinity
temp = gsw.CT_from_t(SA, insitutemp, depth) # computing conservative temperature
gswrho = gsw.rho(SA,temp,P)
timez = time[:,1]

In [17]:
index26= []
depth26= []    #creates an empty list in python so that we can later append values into it
for i in range(temp.shape[1]):      #for all of the values in in temp.shape
    temp_profile= temp[:,i]        #making temp profile, across all depth for each time profile
    index26nonvect = np.argwhere(temp_profile > 26).argmax()    #index of each mixed layer depth
    index26.append(index26nonvect) #list of mixed layer depth
    depth26.append(depth[index26nonvect,i])
depth26=np.array(depth26)
print(depth26.shape)

(3779,)


In [60]:
firstdepthrho03= []
mldrho03 = []
for i in range(rho.shape[1]):
    matcha = np.where(~np.isnan(rho[:,i]))[0] #filters out nan across all rows (:) and across i columns
    firstdepthindex = np.where(~np.isnan(rho[:,i]))[0][0]
    firstrho03 = rho[matcha, i] #extracts non-Nan values
    mlrho03 = firstrho03[0]+.03

    indexmldrho03 = np.argmax(firstrho03 >= mlrho03)
    firstdepthrho03.append(depth[matcha[0],i])
    mldrho03.append(depth[matcha[indexmldrho03], i])

mldrho125 = []
firstdepthrho125 = []
for i in range(rho.shape[1]):
    chai = np.where(~np.isnan(rho[:,i]))[0] #
    firstrho125 = rho[chai, i]
    mlrho125 = firstrho125[0]+.125
    indexmldrho125= np.argmax(firstrho125 >= mlrho125)
    mldrho125.append(depth[chai[indexmldrho125], i])

firstdepth = depth[matcha]

firstrho03 = rho[matcha, i] #extracts non-Nan values
print(firstrho03)

In [10]:
mldrho03 =np.array(mldrho03)
mldrho125 = np.array(mldrho125)
depth26 = np.array(depth26)
timez= time[0,:]

In [11]:
Cp = gsw.cp_t_exact(SA, temp, P) #specific heat capacity
Cpmean= np.nanmean(Cp) # mean specific heat capacity

In [15]:
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.integrate import trapezoid, simpson
eq= rho*temp 
integrated_values_simps = simpson(eq, depth, axis=0) #needed for ocean heat quantity 

In [28]:
heat_total= integrated_values*(Cpmean/1)#/depth26)
heat_bottom_125 = integratednew_values*(Cpmean/1)#(depth26-mldrho125))
heat_top_125 = integratedtoptomld_values*(Cpmean/1)#(mldrho125))

In [72]:
depth0= depth26-(depth[firstdepthindex,1])
deno0 = (Cpmean/depth0)
depth1_125 = mldrho125-(depth[firstdepthindex,1])
deno1_125 = (Cpmean/depth1_125)
depth2_125 = depth26-mldrho125
deno2_125 = (Cpmean/depth2_125)
t = pd.date_range(
    start='2024-05-19T02:30:00',
    end='2024-06-09T05:07:00',
periods=3779)

depth0= depth26-(depth[firstdepthindex,1])
deno0 = (Cpmean/depth0)
depth1_03 = mldrho03-(depth[firstdepthindex,1])
deno1_03 = (Cpmean/depth1_03)
depth2_03 = depth26-mldrho03
deno2_03 = (Cpmean/depth2_03)

In [73]:
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.integrate import simpson
Ktemp = temp + 273
rho26 = rho[index26, :]
rho26mean = np.nanmean(rho26)
rhoxtemp = (temp+(273.15-26)*rho)
integrated_values_simps = simpson(rhoxtemp, x=depth, axis=0)
Cpmean
Cpxrhomean = Cpmean * rho26mean

In [78]:
#calculating from the top to the 26 mark
integrated_values = np.zeros(temp.shape[1]) #idk what this does it just fixes issues

for i in range(temp.shape[1]):
#temp_profile= temp[:,i]
#index26 = np.argwhere(temp_profile1 > 26).argmax()
    xvaluestot = depth[0:index26[i],i]
    data_subsettot = rhoxtemp[0:index26[i], i]
    integrated_value = simpson(data_subsettot, xvaluestot, axis=0)
    integrated_values[i] = integrated_value
print(integrated_value)

integratednew_values = np.zeros(temp.shape[1]) #idk what this does it just fixes issues
#calculating from the mld to the 26 mark
for i in range(temp.shape[1]):
    xvalues26 = depth[0:index26[i],i]
    xvaluesmldrho26125 = depth[indexmldrho125:index26[i],i]
    data_subsetmldrho26125 = rhoxtemp[indexmldrho125:index26[i],i]
    integratednew_value= simpson(data_subsetmldrho26125, xvaluesmldrho26125, axis=0)
    integratednew_values[i] = integratednew_value

#Calculating from the top to the mld

integratedtoptomld_values = np.zeros(temp.shape[1]) #idk what this does it just fixes issues
for i in range(temp.shape[1]):
    xvaluesmldrho125 = depth[0:indexmldrho125,i]
    data_subsetmldrho125 = rhoxtemp[0:indexmldrho125, i]
    integratedtoptomld_value= simpson(data_subsetmldrho125,
    xvaluesmldrho125, axis=0)
    integratedtoptomld_values[i] = integratedtoptomld_value

heat_total = integrated_values*Cpmean#/depth0
heat_bottom_125 = integratednew03_values*Cpmean#/depth1_125
heat_top_125 = integratedtop03_values*Cpmean#/depth2_125

20309394.332123548


In [79]:
#calculating from the mld to the 26 mark for mld 03

integratednew03_values= np.zeros(temp.shape[1]) #idk what this does it just fixes issues

for i in range(temp.shape[1]):
    temp_profile= temp[:,i]
    xvaluesmldrho2603 = depth[indexmldrho03:index26[i],i]
    data_subsetmldrho2603 = rhoxtemp[indexmldrho03:index26[i],i]
    integratednew03_value= simpson(data_subsetmldrho2603, xvaluesmldrho2603, axis=0)
    integratednew03_values[i] = integratednew03_value

#Calculating from the top to the mld
integratedtop03_values= np.zeros(temp.shape[1]) #idk what this does it just fixes issues
for i in range(temp.shape[1]):
    indexmldrho03 = np.argmax(firstrho03 >= mlrho03)
    xvaluesmldrho03 = depth[0:indexmldrho03,i]
    data_subsetmldrho03 = rhoxtemp[0:indexmldrho03, i]
    integratedtop03_value= simpson(data_subsetmldrho03,
    xvaluesmldrho03, axis=0)
    integratedtop03_values[i] = integratedtop03_value

heat_bottom_03 = integratednew03_values*(Cpmean/1)#/depth1_03
heat_top_03 = integratedtop03_values*(Cpmean/1)#/depth2_03

In [81]:
#making heat content into xarrays ni= not interpolated
import xarray as xr
heattotalxr = xr.DataArray(heat_total, coords=[timez], dims=['time']) #turning heat content into xarray arrays
heattop125xr = xr.DataArray(heat_top_125, coords=[timez], dims=['time'])
heatbottom125xr = xr.DataArray(heat_bottom_125, coords=[timez], dims=['time'])

interp_times = pd.date_range('2024-05-19T02:30:00',
'2024-06-09T05:07:00', freq='10s')
heat_total_xr= heattotalxr.interp(time=t)
heat_top_125_xr = heattop125xr.interp(time=t)
heat_bottom_125_xr = heatbottom125xr.interp(time=t)

heat_total_125_roll= heat_total_xr.rolling(time=170, min_periods=1, center=True).mean() 
#daily mean (140= 3779/22 days), with it starting at 1, and is centered on the day
heat_top_125_roll = heat_top_125_xr.rolling(time=170, min_periods=1,center=True).mean()
heat_bottom_125_roll = heat_bottom_125_xr.rolling(time=170, min_periods=1,center=True).mean()

ModuleNotFoundError: No module named 'xarray'